# Min-Max Tree ile Kelime Oyunu
* Burada oluşturulan ağacın her düğümünün, çocuk düğümlerden oluşacak diziyi tutmak için bir alanı, kelime tutmak için bir alanı ve yerine göre kullanımı farklılık gösterecek bir skor sayısı tutacak alanı bulunmaktadır.

In [1]:
class tree:
    def __init__(self, root_word, score = 0, children = None):
        self.root_word = root_word
        self.score = score
        self.children = children

## Skorlama Fonksiyonu
* Burada harflere atanan puanlar Kelimelik isimli oyunda harflere atanan puanlar ile aynı olarak alınmıştır.
* Kelimedeki her harfin puanı toplanarak o kelimenin skoru elde edilir.

In [2]:
scores = {'a': 1,'b':3,'c':4,'ç':4,'d':3,
          'e':1,'f':7,'g':5,'ğ':8,'h':5,
          'ı':2,'i':1,'j':10,'k':1,'l':1,
          'm':2,'n':1,'o':2,'ö':7,'p':5,
          'r':1,'s':2,'ş':4,'t':1,'u':2,
          'ü':3,'v':7,'y':3,'z':4}

def scoring(word):
    score = 0
    for i in word:
        score += scores[i]
    return score

* Önceden oluşturulan sözlük (turkish_dict.pkl) burada sözlük yapısını tutacak **words_dict** isimli değişkenin içine okunur.

In [3]:
import pickle

a_file = open("turkish_dict.pkl", "rb")
words_dict = pickle.load(a_file)

### Sözlük Maskeleme ile Zorluk Seçimi
* Bu fonksiyonda kullanıcı tarafınfan seçilen zorluk oranı ile ters orantılı olarak ajanın kelimelere bakabileceği sözlük geçici olarak maskelenmektedir.
* Sözlük maskelenmesi hem oyunda çeşitlilik oluşturmakta hem de kullanıcı tecrübesini iyileştirmektedir. 

In [4]:
import numpy as np

def mask_vocab(list_of_words, difficulty):
    
    len_list = len(list_of_words)
    
    if difficulty == 'very easy':
        return np.random.choice(list_of_words, int(len_list * 0.2))
    elif difficulty == 'easy':
        return np.random.choice(list_of_words, int(len_list * 0.4))
    elif difficulty == 'medium':
        return np.random.choice(list_of_words, int(len_list * 0.6))
    elif difficulty == 'hard':
        return np.random.choice(list_of_words, int(len_list * 0.8))
    elif difficulty == 'very hard':
        return np.random.choice(list_of_words, int(len_list * 0.99))

In [5]:
a = mask_vocab(words_dict['a'], 'very easy')

In [6]:
len(a)

915

## Ağaç Oluşturma Fonksiyonu
* Öncelikle bir düğüm oluşturulur ve parametre olarak gönderilen kelime bu düğüme atanır.
* Ardından bu düğümün çocuk düğümleri tek tek oluşturulur. Bu çocuk düğümlerin sayısı sözlükte ana düğümün son harfiyle başlayan kelime sayısı kadardır.
* Çocukların tuttukları kelimeler ise yine ana düğümün son harfiyle başlayan kelimelerdir.
* 'ğ' harfi ile biten kelimelerin çocuk düğümleri 'g' harfi ile başlayan kelimelerden oluşturulmaktadır.
* Düğümlerin skor alanlarında tutulacak bilgiler ise kelimenin kendi skoru değil çocuk düğümleri arasındaki en büyük skora sahip kelimenin skorudur.
* Aşağıdaki çizimde bu yapı daha ayrıntılı anlatılmıştır.

In [7]:
def create_tree(root_word, difficulty):
    
    if difficulty == 1:
        difficulty = 'very easy'
    elif difficulty == 2:
        difficulty = 'easy'    
    elif difficulty == 3:
        difficulty = 'medium'    
    elif difficulty == 4:
        difficulty = 'hard'
    elif difficulty == 5:
        difficulty = 'very hard'
    
    root = tree(root_word)
    children = []
    
    letter_for_dict = 'x'
    
    if root_word[-1] == 'ğ':
        letter_for_dict = 'g'
    else:
        letter_for_dict = root_word[-1]   
    
    masked_words = mask_vocab(words_dict[letter_for_dict], difficulty)
    
    for word in masked_words:
        tmp_node = tree(word)
        
        letter_for_dict_2 = 'x'
        if word[-1] == 'ğ':
            letter_for_dict_2 = 'g'
        else:
            letter_for_dict_2 = word[-1]
            
        maximum = 0
        for w in words_dict[letter_for_dict_2]:
            score_w = scoring(w)
            if score_w > maximum:
                maximum = score_w
        tmp_node.score = maximum 
    
        children.append(tmp_node)
        
    root.children = children
    return root

<img src="minmax_tree.png">

In [8]:
root = create_tree('toprak', 5)

## Ajanın Kelime Seçimi
* Buradaki fonksiyonu kullanarak kelime seçen ajan, kelimeleri seçerken rakibinin **minimum puan** alabileceği harflerle biten ve kendisinin de **maksimum puan** alabileceği kelimeyi seçmektedir.

In [9]:
def choose_best_word(root, forbidden_words, max_len):
    min_score = root.children[0].score
    min_index = 0
    max_word = 'a'

    for i, child_node in enumerate(root.children):
        if child_node.score <= min_score:
            min_score = child_node.score
            min_index = i
            
            if (scoring(child_node.root_word) > scoring(max_word)) and (child_node.root_word not in forbidden_words) and len(child_node.root_word) <= max_len:
                max_word = child_node.root_word
    return max_word

* max_len parametresi oyunun ikinci versiyonunda kullanılacak bir parametredir.

In [10]:
choose_best_word(root, [], 100) 

'kumrugöğsü'

## İlk Oyun (v1.0)
* Bu oyun geliştirilmekte olan oyunun ilk versiyonudur ve kelime tahtası bulunmamaktadır.
* Burada kullanıcının yazdığı kelime ile oyun başlar. Girilen kelimenin sözlükte bulunup bulunmadığı kontrol edilir ve varsa sıra ajana geçer.
* Ajan, kullanıcının girdiği kelimenin son harfiyle başlayan, sözlükte bulunan, kendisine **en çok** puanı kazandıracak ve yazdığı kelimenin son harfini rakibinin **en az** puan kazanabileceği şekilde olduğu kelimeyi seçmektedir.
* Hem oyuncu hem de ajan daha önce kullanılmış bir kelimeyi de seçememektedir.
* Parametre olarak gönderilen puana ilk ulaşan kazanır.

In [11]:
def first_game(max_score = 200):
    toggle = 1 # 0 - user // 1 - agent
    
    forbidden_words = []
    
    user_word = input('İlk kelimeyi giriniz: ')
    if user_word == '.':
        print("Çıkış yapılıyor...")
        return
    while user_word not in words_dict[user_word[0]]:
        print("Sözlükte bulunan bir kelime giriniz...")
        user_word = input('İlk kelimeyi giriniz: ')
    difficulty = 4
    user_score = scoring(user_word)
    print("Skorunuz : ", user_score)
    forbidden_words.append(user_word)
    agent_score = 0
    print("-"*50)
    while(True):
        if toggle == 1:
            root = create_tree(user_word, difficulty)
            agent_word = choose_best_word(root, forbidden_words, 100)
            print("Ajanın Kelimesi : ", agent_word)
            agent_score += scoring(agent_word)
            toggle = 0
            print("Ajanın puanı : ", agent_score)
            forbidden_words.append(agent_word)
            if agent_score >= max_score:
                print("Ajan kazandı :D")
                return
        print("-"*50)
        if toggle == 0:
            #root = create_tree(user_word)
            #user_word = choose_best_word(root)
            
            user_word = input('Kelimeniz: ')
            if user_word == '.':
                print("Çıkış yapılıyor...")
                return
            #print(user_word not in words_dict[user_word[0]])
            while (user_word not in words_dict[user_word[0]]) or (user_word[0] != agent_word[-1]) or (user_word in forbidden_words):
                if (user_word[0] != agent_word[-1]):
                    print("Son kelimenin son harfiyle başlayan bir kelime giriniz!")
                if (user_word not in words_dict[user_word[0]]):
                    print("Sözlükte bulunan bir kelime giriniz!")
                if (user_word in forbidden_words):
                    print("Daha önce kullanılmayan bir kelime giriniz!")
                
                #print(user_word[0], agent_word[-1], user_word)
                user_word = input('Kelimeniz: ')
                if user_word == '.':
                    print("Çıkış yapılıyor...")
                    return
            forbidden_words.append(user_word)
            user_score += scoring(user_word)
            toggle = 1
            print("Skorunuz : ", user_score)
            if user_score >= max_score:
                print("Siz kazandınız.")
                return
        print("-"*50)

* Oyundan çıkış yapmak için '.' karakterini giriniz.

In [14]:
first_game(max_score = 200)

İlk kelimeyi giriniz:  kalem


Skorunuz :  6
--------------------------------------------------
Ajanın Kelimesi :  mizanpajlı
Ajanın puanı :  28
--------------------------------------------------


Kelimeniz:  ıspanak


Skorunuz :  19
--------------------------------------------------
Ajanın Kelimesi :  katranköpüğü
Ajanın puanı :  61
--------------------------------------------------


Kelimeniz:  üfürmek


Skorunuz :  37
--------------------------------------------------
Ajanın Kelimesi :  kumrugöğsü
Ajanın puanı :  94
--------------------------------------------------


Kelimeniz:  ülke


Skorunuz :  43
--------------------------------------------------
Ajanın Kelimesi :  ebekuşağı
Ajanın puanı :  117
--------------------------------------------------


Kelimeniz:  .


Çıkış yapılıyor...


## Kelime Tahtası ile Oyun (v2.0)

* Verilen boyutlarda kare bir oyun tahtası oluşturur.

In [15]:
def make_board(size):
    return [[" " for i in range(size)] for j in range(size)]

* Tahtayı ve oyuncuların skorlarını ekrana bastırır.

In [16]:
def print_board(board,user_score, agent_score):
    print("-"*50)
    print("Sizin Skorunuz : ", user_score)
    print("Ajanın Skoru   : ", agent_score)
    print("-"*50)
    for j,row in enumerate(board):
        print(j,end="\t")
        for i,square in enumerate(row):
            if square == "☐":
                print(square, end = " ")
                row[i] = " "
            elif square != " ":
                print(square, end= " "),
            else:
                print('.', end= " "),
        print("")
    print("-"*50)

### Kelimenin Tahtaya Yerleştirilmesi
* Oyun tahtası, tahtanın boyutu, tahtaya yazılacak kelime, kelimenin yazılmaya başlanacağı koordinat ve hangi yöne doğru yazılacağı bilgileri parametre olarak fonksiyona gönderilir.
* Başlangıç noktaları tahtanın dışına tekabül ediyorsa hata verilir.
* Seçilen yönde kelimenin önünde başka bir kelime varsa veya seçilen yönde yeterli boşluk yoksa hata verilir.
* Eğer hata yoksa kelime tahtaya verilen başlangıç noktasından verilen yöne doğru yazılır.

In [17]:
def put_board(board, board_size, word, start_x, start_y, direction):
    
    if start_x > board_size-1 or start_y > board_size-1:
        print("Tahtanın dışına kelime yazamazsınız!")
        return False,0,0
    
    
    len_word = len(word)
    if direction == 'right':
        if (start_y + len_word) > board_size:
            print("Kelime tahtaya sığmadı!")
            return False,0,0
        
        for i in range(len_word):
            if board[start_x][start_y + i] != ' ':
                print("Kelime diğer kelimelerden biriyle çakıştı!")
                return False,0,0
        for i in range(len_word):
            board[start_x][start_y + i] = word[i]
            
        return True, start_x, start_y+len_word-1
    
    if direction == 'left':
        if (start_y - len_word) < -1:
            print("Kelime tahtaya sığmadı!")
            return False,0,0
        
        for i in range(len_word):
            if board[start_x][start_y - i] != ' ':
                print("Kelime diğer kelimelerden biriyle çakıştı!")
                return False,0,0 
        for i in range(len_word):
            board[start_x][start_y - i] = word[i]
        
        return True, start_x, start_y-len_word+1
    
    if direction == 'up':
        if (start_x - len_word) < -1:
            print("Kelime tahtaya sığmadı!")
            return False,0,0
        
        for i in range(len_word):
            if board[start_x - i][start_y] != ' ':
                print("Kelime diğer kelimelerden biriyle çakıştı!")
                return False,0,0 
        for i in range(len_word):
            board[start_x - i][start_y] = word[i]
        
        return True, start_x-len_word+1, start_y 
    
    if direction == 'down':
        if (start_x + len_word) > board_size:
            print("Kelime tahtaya sığmadı!")
            return False,0,0
        
        for i in range(len_word):
            if board[start_x + i][start_y] != ' ':
                print("Kelime diğer kelimelerden biriyle çakıştı!")
                return False,0,0 
        for i in range(len_word):
            board[start_x + i][start_y] = word[i]
        
        return True, start_x+len_word-1, start_y    

In [18]:
board_size = 20
board = make_board(board_size)

#### Kullanım denemeleri : 

In [19]:
put_board(board, board_size, 'ailecek', 2,1,'right')
put_board(board, board_size, 'ailecek', 8,3,'down')

(True, 14, 3)

In [20]:
print_board(board,0,0)

--------------------------------------------------
Sizin Skorunuz :  0
Ajanın Skoru   :  0
--------------------------------------------------
0	. . . . . . . . . . . . . . . . . . . . 
1	. . . . . . . . . . . . . . . . . . . . 
2	. a i l e c e k . . . . . . . . . . . . 
3	. . . . . . . . . . . . . . . . . . . . 
4	. . . . . . . . . . . . . . . . . . . . 
5	. . . . . . . . . . . . . . . . . . . . 
6	. . . . . . . . . . . . . . . . . . . . 
7	. . . . . . . . . . . . . . . . . . . . 
8	. . . a . . . . . . . . . . . . . . . . 
9	. . . i . . . . . . . . . . . . . . . . 
10	. . . l . . . . . . . . . . . . . . . . 
11	. . . e . . . . . . . . . . . . . . . . 
12	. . . c . . . . . . . . . . . . . . . . 
13	. . . e . . . . . . . . . . . . . . . . 
14	. . . k . . . . . . . . . . . . . . . . 
15	. . . . . . . . . . . . . . . . . . . . 
16	. . . . . . . . . . . . . . . . . . . . 
17	. . . . . . . . . . . . . . . . . . . . 
18	. . . . . . . . . . . . . . . . . . . . 
19	. . . . . . . . . . . . . . .

In [21]:
put_board(board, board_size, 'ailecek', 20,20,'right')

Tahtanın dışına kelime yazamazsınız!


(False, 0, 0)

In [22]:
put_board(board, board_size, 'ailecek', 10,15,'right')

Kelime tahtaya sığmadı!


(False, 0, 0)

In [23]:
put_board(board, board_size, 'ai', 17,13,'down')

(True, 18, 13)

In [24]:
put_board(board, board_size, 'aile', 19,3,'left')

(True, 19, 0)

In [25]:
print_board(board,0,0)

--------------------------------------------------
Sizin Skorunuz :  0
Ajanın Skoru   :  0
--------------------------------------------------
0	. . . . . . . . . . . . . . . . . . . . 
1	. . . . . . . . . . . . . . . . . . . . 
2	. a i l e c e k . . . . . . . . . . . . 
3	. . . . . . . . . . . . . . . . . . . . 
4	. . . . . . . . . . . . . . . . . . . . 
5	. . . . . . . . . . . . . . . . . . . . 
6	. . . . . . . . . . . . . . . . . . . . 
7	. . . . . . . . . . . . . . . . . . . . 
8	. . . a . . . . . . . . . . . . . . . . 
9	. . . i . . . . . . . . . . . . . . . . 
10	. . . l . . . . . . . . . . . . . . . . 
11	. . . e . . . . . . . . . . . . . . . . 
12	. . . c . . . . . . . . . . . . . . . . 
13	. . . e . . . . . . . . . . . . . . . . 
14	. . . k . . . . . . . . . . . . . . . . 
15	. . . . . . . . . . . . . . . . . . . . 
16	. . . . . . . . . . . . . . . . . . . . 
17	. . . . . . . . . . . . . a . . . . . . 
18	. . . . . . . . . . . . . i . . . . . . 
19	e l i a . . . . . . . . . . .

### Maksimum Alan Bulma
* Tahtaya yazılan kelimenin ardından, kelimenin bittiği koordinatlar ve tahtanın kendisi fonksiyona gönderilir.
* Bitiş noktasından itibaren 4 yöne doğru bir kontrol yapar ve her yönde yazılabilecek kelime için ne kadar boşluk olduğu bilgisini döner.
* Maksimum boşluğun hangi yöne doğru olduğunu ve kaç boşluk bulunduğu bilgisini de ajanın kullanımı için döner. 

In [26]:
def max_space(board, board_size, x,y):
    max_space = 0
    direction = 'x'
    
    each = []
    
    count = 1
    for i in range(x+1,board_size):
        if board[i][y] == ' ':
            count += 1
        else:
            break
    #print("down : ", count)
    each.append(count)
    if count > max_space:
        max_space = count
        direction = 'down'
    
    count = 1
    ctrl = 0
    for i in range(x-1,0,-1):
        
        if board[i][y] == ' ':
            count += 1
        else:
            ctrl = 1
            break
    if ctrl == 0:
        count += 1
            
    #print("up : ", count)       
    each.append(count)
    if count > max_space:
        max_space = count
        direction = 'up'
    
    count = 1
    for i in range(y+1,board_size):
        if board[x][i] == ' ':
            count += 1
        else:
            break
    #print("right : ", count)        
    each.append(count)
    if count > max_space:
        max_space = count
        direction = 'right'
    
    count = 1
    ctrl = 0
    for i in range(y-1,0,-1):
        if board[x][i] == ' ':
            count += 1
        else:
            ctrl = 1
            break
    if ctrl == 0:
        count += 1
        
    #print("left : ", count)
    each.append(count)
    if count > max_space:
        max_space = count
        direction = 'left'
        
    return direction, max_space, each

In [27]:
print_board(board,0,0)

--------------------------------------------------
Sizin Skorunuz :  0
Ajanın Skoru   :  0
--------------------------------------------------
0	. . . . . . . . . . . . . . . . . . . . 
1	. . . . . . . . . . . . . . . . . . . . 
2	. a i l e c e k . . . . . . . . . . . . 
3	. . . . . . . . . . . . . . . . . . . . 
4	. . . . . . . . . . . . . . . . . . . . 
5	. . . . . . . . . . . . . . . . . . . . 
6	. . . . . . . . . . . . . . . . . . . . 
7	. . . . . . . . . . . . . . . . . . . . 
8	. . . a . . . . . . . . . . . . . . . . 
9	. . . i . . . . . . . . . . . . . . . . 
10	. . . l . . . . . . . . . . . . . . . . 
11	. . . e . . . . . . . . . . . . . . . . 
12	. . . c . . . . . . . . . . . . . . . . 
13	. . . e . . . . . . . . . . . . . . . . 
14	. . . k . . . . . . . . . . . . . . . . 
15	. . . . . . . . . . . . . . . . . . . . 
16	. . . . . . . . . . . . . . . . . . . . 
17	. . . . . . . . . . . . . a . . . . . . 
18	. . . . . . . . . . . . . i . . . . . . 
19	e l i a . . . . . . . . . . .

In [28]:
max_space(board=board, board_size=20, x=17, y=13)

('up', 18, [1, 18, 7, 14])

### Oyunun Oynandığı Fonksiyon

#### Oyun İşleyişi :
* Başlangıçta oyun kullanıcıya tahta üzerinde rastgele bir nokta atar ve oyunun bu noktadan başlaması zorunludur.
* Oyuncu sözlükte olan ve seçtiği yönde tahtaya sığabilecek bir kelimeyle oyunu başlatır.
* Ajan oyuncunun yazdığı kelimenin son harfiyle başlayan bir kelime seçer. 
* Bu kelimeyi seçerken öncelikle tahtada gidebileceği yönlerden en boş olanı bulur ve kelimeyi ona göre seçer.
* Aynı zamanda kelime seçimi bir önceki versiyonda olduğu gibi **min-max tree** kullanılarak yapılmaktadır.
* Ardından oyuncu ajanın yazdığı kelimenin son harfiyle başlayan, oyun tahtasına sığabilecek ve oyunun bir önceki versiyonundaki kontrollerden geçen bir kelimeyi seçer ve tahtaya istediği yönde yazar.
* Oyun kullanıcı '.' karakterini girene kadar veya tahtada gidilecek yer kalmayana kadar veya kullanıcı kelime seçerken 5 başarısız deneme yaparsa biter.

In [29]:
import random

def game_with_board(board_size):
    print("-"*50)
    print("Tahta Üzerinde Kelimelik Oyununa Hoş Geldiniz!")
    print("- Kelimeleriniz Türkçe olmalıdır.")
    print("- İlk tur hariç karşı tarafın yazdığı kelimenin bittiği yerden itibaren, tahtanın boş olan kısmına\n'aşağı', 'yukarı', 'sağ', 'sol' yönlerinde kelimeler yazabilirsiniz.")
    print("- Başarısız deneme hakkınız 5'tir, bunun üzerinde deneme yaparsanız oyun sonlanır.")
    print("- Oyundan çıkmak için sıra sizdeyken '.' karakterini girmeniz gerekmektedir.")
    print("-"*50)
    print("\nBaşlamak için herhangi bir tuşa basınız...")
    input()
    
    #clear_output(wait=True)
    
    print("Bir zorluk seviyesi seçiniz : ")
    print("1 -> Çok Kolay\n2 -> Kolay\n3 -> Orta\n4 -> Zor\n5 -> TDK'de Çalışıyorum")
    difficulty = int(input())
    while difficulty not in [1,2,3,4,5]:
        print("Geçersiz zorluk düzeyi seçimi! Tekrar seçiniz...")
        difficulty = int(input())
    
    #clear_output(wait=True)
    
    board = make_board(board_size)
    initial_x = random.randint(0,board_size-1)
    initial_y = random.randint(0,board_size-1)
    board[initial_x][initial_y] = '☐'
    
    user_score = 0
    agent_score = 0
    print_board(board, user_score, agent_score)
    
    toggle = 1 # 0 - user | 1 - agent
    
    forbidden_words = []
    user_trying = 1
    user_word = input('İlk kelimeyi giriniz: ')
    if user_word == '.':
        print("Çıkış yapılıyor...")
        return
        
    while (user_word[0] not in words_dict.keys()) or (user_word not in words_dict[user_word[0]]):
        if (user_word[0] not in words_dict.keys()):
            print("Türkçe karakterler kullanınız!")
            user_word = input('İlk kelimeyi giriniz: ')
        elif (user_word not in words_dict[user_word[0]]):
            print("Sözlükte bulunan bir kelime giriniz...")
            user_word = input('İlk kelimeyi giriniz: ')
        if user_word == '.':
            print("Çıkış yapılıyor...")
            return
        user_trying += 1
        if user_trying >= 5:
            print('Deneme Hakkınız ve Oyun Bitti!')
            if agent_score>=user_score:
                print("Ajan Kazandı!")
            else:
                print("Siz Kazandınız!")
            return

    ctrl_dir = False
    while ctrl_dir == False:
        direction  = input("Yön seçiniz : ")
        if direction == "sağ":
            ctrl_dir,last_x,last_y = put_board(board,board_size,user_word, initial_x, initial_y, 'right')
        elif direction == "sol":
            ctrl_dir,last_x,last_y = put_board(board,board_size,user_word, initial_x, initial_y, 'left')
        elif direction == "yukarı":
            ctrl_dir,last_x,last_y = put_board(board,board_size,user_word, initial_x, initial_y, 'up')
        elif direction == "aşağı":
            ctrl_dir,last_x,last_y = put_board(board,board_size,user_word, initial_x, initial_y, 'down')
        else:
            print("Yön olarak aşağı, yukarı, sağ veya sol giriniz!")
    
    user_score = scoring(user_word)
    forbidden_words.append(user_word)
    agent_score = 0
    #clear_output(wait=True)
    print_board(board, user_score, agent_score)
    
    while(True):
        if toggle == 1:
            print("Ajan düşünüyor...")
            root = create_tree(user_word, difficulty)
            #print(last_x,last_y)
            direction, max_len, each = max_space(board,board_size, last_x, last_y)
            agent_word = choose_best_word(root, forbidden_words, max_len)
            
            if agent_word == 'a':
                print('Ajanı Köşeye Sıkıştırdınız!')
                if agent_score>=user_score:
                    print("Ajan Kazandı!")
                else:
                    print("Siz Kazandınız!")                
                return
            
            print("Ajanın Kelimesi : ", agent_word)
            
            if direction == 'right':
                _,last_x,last_y = put_board(board,board_size,agent_word[1:], start_x=last_x, start_y=last_y+1,direction=direction)
            elif direction == 'left':
                _,last_x,last_y = put_board(board,board_size,agent_word[1:], start_x=last_x, start_y=last_y-1,direction=direction)
            elif direction == 'up':
                _,last_x,last_y = put_board(board,board_size,agent_word[1:], start_x=last_x-1, start_y=last_y,direction=direction)
            elif direction == 'down':
                _,last_x,last_y = put_board(board,board_size,agent_word[1:], start_x=last_x+1, start_y=last_y,direction=direction)
            
            agent_score += scoring(agent_word)
            toggle = 0
            #clear_output(wait=True)
            print_board(board,user_score,agent_score)
            
            forbidden_words.append(agent_word)
            
            if len([i for i in each if i < 2]) == 4:
                print('Oyun Bitti!')
                if agent_score>=user_score:
                    print("Ajan Kazandı!")
                else:
                    print("Siz Kazandınız!")
                return
        
        _, max_len, _= max_space(board,board_size, last_x, last_y)
        user_trying = 1
        if toggle == 0:            
            user_word = input('Kelimeniz: ')
            if user_word == '.':
                print("Çıkış yapılıyor...")
                return
            #print(user_word not in words_dict[user_word[0]])
            while (user_word[0] not in words_dict.keys()) or (user_word not in words_dict[user_word[0]]) or (user_word[0] != agent_word[-1]) or (user_word in forbidden_words) or (len(user_word) > max_len):
                if (user_word[0] not in words_dict.keys()):
                    print("Türkçe karakterler kullanınız!")
                elif (user_word[0] != agent_word[-1]):
                    print("Son kelimenin son harfiyle başlayan bir kelime giriniz!")
                elif (user_word not in words_dict[user_word[0]]):
                    print("Sözlükte bulunan bir kelime giriniz!")
                elif (user_word in forbidden_words):
                    print("Daha önce kullanılmayan bir kelime giriniz!")
                elif (len(user_word) > max_len):
                    print("Bu kelime hiçbir yönde yazılamaz!")
                user_trying += 1
                if user_trying >= 5:
                    print('Deneme Hakkınız ve Oyun Bitti!')
                    if agent_score>=user_score:
                        print("Ajan Kazandı!")
                    else:
                        print("Siz Kazandınız!")
                    return
                user_word = input('Kelimeniz: ')
                if user_word == '.':
                    print("Çıkış yapılıyor...")
                    return
            #print("Last X = ",last_x, "Last Y = ", last_y)
            ctrl_dir = False
            while ctrl_dir == False:
                direction  = input("Yön seçiniz : ")
                if direction == "sağ":
                    ctrl_dir,last_x_tmp,last_y_tmp = put_board(board,board_size,user_word[1:], last_x, last_y+1, 'right')
                elif direction == "sol":
                    ctrl_dir,last_x_tmp,last_y_tmp = put_board(board,board_size,user_word[1:], last_x, last_y-1, 'left')
                elif direction == "yukarı":
                    ctrl_dir,last_x_tmp,last_y_tmp = put_board(board,board_size,user_word[1:], last_x-1, last_y, 'up')
                elif direction == "aşağı":
                    ctrl_dir,last_x_tmp,last_y_tmp = put_board(board,board_size,user_word[1:], last_x+1, last_y, 'down')
                else:
                    print("Yön olarak aşağı, yukarı, sağ veya sol giriniz!")

            forbidden_words.append(user_word)
            user_score += scoring(user_word)
            toggle = 1
            
            #clear_output(wait=True)
            print_board(board, user_score, agent_score)
            last_x, last_y = last_x_tmp, last_y_tmp
            if len([i for i in each if i < 2]) == 4:
                print('Oyun Bitti!')
                if agent_score>user_score:
                    print("Ajan Kazandı!")
                else:
                    print("Siz Kazandınız!")
                return

In [31]:
game_with_board(board_size=20)

--------------------------------------------------
Tahta Üzerinde Kelimelik Oyununa Hoş Geldiniz!
- Kelimeleriniz Türkçe olmalıdır.
- İlk tur hariç karşı tarafın yazdığı kelimenin bittiği yerden itibaren, tahtanın boş olan kısmına
'aşağı', 'yukarı', 'sağ', 'sol' yönlerinde kelimeler yazabilirsiniz.
- Başarısız deneme hakkınız 5'tir, bunun üzerinde deneme yaparsanız oyun sonlanır.
- Oyundan çıkmak için sıra sizdeyken '.' karakterini girmeniz gerekmektedir.
--------------------------------------------------

Başlamak için herhangi bir tuşa basınız...


Bir zorluk seviyesi seçiniz : 
1 -> Çok Kolay
2 -> Kolay
3 -> Orta
4 -> Zor
5 -> TDK'de Çalışıyorum


 2


--------------------------------------------------
Sizin Skorunuz :  0
Ajanın Skoru   :  0
--------------------------------------------------
0	. . . . . . . . . . . . . . . . . . . . 
1	. . . . . ☐ . . . . . . . . . . . . . . 
2	. . . . . . . . . . . . . . . . . . . . 
3	. . . . . . . . . . . . . . . . . . . . 
4	. . . . . . . . . . . . . . . . . . . . 
5	. . . . . . . . . . . . . . . . . . . . 
6	. . . . . . . . . . . . . . . . . . . . 
7	. . . . . . . . . . . . . . . . . . . . 
8	. . . . . . . . . . . . . . . . . . . . 
9	. . . . . . . . . . . . . . . . . . . . 
10	. . . . . . . . . . . . . . . . . . . . 
11	. . . . . . . . . . . . . . . . . . . . 
12	. . . . . . . . . . . . . . . . . . . . 
13	. . . . . . . . . . . . . . . . . . . . 
14	. . . . . . . . . . . . . . . . . . . . 
15	. . . . . . . . . . . . . . . . . . . . 
16	. . . . . . . . . . . . . . . . . . . . 
17	. . . . . . . . . . . . . . . . . . . . 
18	. . . . . . . . . . . . . . . . . . . . 
19	. . . . . . . . . . . . . . .

İlk kelimeyi giriniz:  saman
Yön seçiniz :  sol


...

--------------------------------------------------
Sizin Skorunuz :  29
Ajanın Skoru   :  44
--------------------------------------------------
0	. . . . . . . . . . . . . . . . . . . . 
1	. n a m a s . . . . . . . . . . . . . . 
2	. a . . . . . . . . . . . . . . . . . . 
3	. m . . . . . . . . . . . . . . . . . . 
4	. a . . . . . . . . . . . . . . . . . . 
5	. z . . . . . . . . . . . . . . . . . . 
6	. l . . . . . . . . . . . . . . . . . . 
7	. a . . . . . . . . . . . . . . . . . . 
8	. ğ . . . . . . . . . . . . . . . . . . 
9	. ı l ı m a n i s a n b a l ı ğ ı . . . 
10	. . . . . . . . . . . . . . . . s . . . 
11	. . . . . . . . . . . . . . . . p . . . 
12	. . . . . . . . . . . . . . . . a . . . 
13	. . . . . . . . . . . . . . . . n . . . 
14	. . . . . . . . . . . . . . . . a . . . 
15	. . . . . . . . . . . . . . . . k . . . 
16	. . . . . . . . . . . . . . . . . . . . 
17	. . . . . . . . . . . . . . . . . . . . 
18	. . . . . . . . . . . . . . . . . . . . 
19	. . . . . . . . . . . . . .

Kelimeniz:  üstüpü
Yön seçiniz :  sol


--------------------------------------------------
Sizin Skorunuz :  46
Ajanın Skoru   :  77
--------------------------------------------------
0	. . . . . . . . . . . . . . . . . . . . 
1	. n a m a s . . . . . . . . . . . . . . 
2	. a . . . . . . . . . . . . . . . . . . 
3	. m . . . . . . . . . . . . . . . . . . 
4	. a . . . . . . . . . . . . . . . . . . 
5	. z . . . . . . . . . . . . . . . . . . 
6	. l . . . . . . . . . . . . . . . . . . 
7	. a . . . . . . . . . . . . . . . . . . 
8	. ğ . . . . . . . . . . . . . . . . . . 
9	. ı l ı m a n i s a n b a l ı ğ ı . . . 
10	. . . . . . . . . . . . . . . . s . . . 
11	. . . . . . . . . . . . . . . . p . . . 
12	. . . . . . . . . . . . . . . . a . . . 
13	. . . . . . . . . . . . . . . . n . . . 
14	. . . . . . . . . . . . . . . . a . . . 
15	ü p ü t s ü ğ ü p ö k n a r t a k . . . 
16	. . . . . . . . . . . . . . . . . . . . 
17	. . . . . . . . . . . . . . . . . . . . 
18	. . . . . . . . . . . . . . . . . . . . 
19	. . . . . . . . . . . . . .

Kelimeniz:  ıstakoz
Yön seçiniz :  yukarı


--------------------------------------------------
Sizin Skorunuz :  59
Ajanın Skoru   :  103
--------------------------------------------------
0	. . . . . . . . . . . . . . . . . . . . 
1	z n a m a s . . . . . . . . . . . . . . 
2	o a . . . . . . . . . . . . . . . . . . 
3	k m . . . . . . . . . . . . . . . . . . 
4	a a . . . . . . . . . . . . . . . . . . 
5	t z . . . . . . . . . . . . . . . . . . 
6	s l . . . . . . . . . . . . . . . . . . 
7	ı a . . . . . . . . . . . . . . . . . . 
8	ç ğ . . . . . . . . . . . . . . . . . . 
9	t ı l ı m a n i s a n b a l ı ğ ı . . . 
10	ı . . . . . . . . . . . . . . . s . . . 
11	ğ . . . . . . . . . . . . . . . p . . . 
12	a . . . . . . . . . . . . . . . a . . . 
13	k . . . . . . . . . . . . . . . n . . . 
14	ç . . . . . . . . . . . . . . . a . . . 
15	ü p ü t s ü ğ ü p ö k n a r t a k . . . 
16	. . . . . . . . . . . . . . . . . . . . 
17	. . . . . . . . . . . . . . . . . . . . 
18	. . . . . . . . . . . . . . . . . . . . 
19	. . . . . . . . . . . . . 

* Ajan veya oyuncu köşeye sıkıştığında veya yazılabilecek kelime kalmadığında oyunu puanı çok olan kazanmaktadır.